### **lightning ai 4 cpu 기반**
- **1. 년도별_상위출현단어처럼 차수별 상위출현단어, 전체년도 합쳐서 상위출현단어 볼 수 있는지 여쭤보기**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import time
import gzip
import pickle
import os

start_time = time.time()

# 현재 디렉토리를 확인합니다.
current_dir = '/content/drive/MyDrive/이대과제/data_gzip'

# 파일 리스트를 가져옵니다.
file_list = [f for f in os.listdir(current_dir) if f.endswith('.pkl.gz')]

# 빈 데이터프레임 리스트를 생성합니다.
df_list = []

# 각 gzip 파일을 읽어 데이터프레임으로 변환하고 리스트에 추가합니다.
for file in file_list:
    try:
        with gzip.open(os.path.join(current_dir, file), 'rb') as f:
            data = pickle.load(f)
            df = pd.DataFrame(data)
            if not df.empty:
                df_list.append(df)
            else:
                print(f"Empty DataFrame in file: {file}")
    except Exception as e:
        print(f"Error reading file {file}: {e}")

# 빈 리스트가 아닌지 확인합니다.
if df_list:
    # 모든 데이터프레임을 하나로 합칩니다.
    df = pd.concat(df_list, ignore_index=True)
else:
    print("No valid DataFrames to concatenate.")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Elapsed time: 58.23 seconds


In [3]:
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
df = df[~df.duplicated(['일자', '언론사', '제목'])]

df= df[['일자', '특성추출(가중치순 상위 50개)']]

# df['일자']를 날짜로 변경하고 index로 지정
df['일자'] = pd.to_datetime(df['일자'], format='%Y%m%d')
df['년도']=df['일자'].apply(lambda x: x.year)
df['월']=df['일자'].apply(lambda x: x.month)

### 불용어

In [6]:
data = pd.read_excel('/content/drive/MyDrive/이대과제/data/불용어후보.xlsx')
print(data.shape)
data
stop_words = list(data[data['불용어_0622'] == 1]['단어'].tolist())

(4837, 6)


In [7]:
# corpus 정제
import re

def preprocess_corpus(corpus):
    updated_corpus = []
    pattern_mixed = re.compile(r'(?=.*\d)(?=.*[a-zA-Z가-힣])')
    pattern_korean_english = re.compile(r'^[가-힣a-zA-Z]+$')

    for document in corpus:
        if isinstance(document, str):
            words = document.split(',')
            filtered_words = [word for word in words if pattern_korean_english.match(word) and not pattern_mixed.search(word)]
            updated_corpus.append(','.join(filtered_words))
        else:
            updated_corpus.append('')
    return updated_corpus


## **1. 년도별 출현단어**

In [8]:
from tqdm import tqdm_notebook
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer # Import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


# 시작년도, 끝년도 지정
start_year = 2010
end_year = 2024

corpus = [""] * len(df)

result_df = pd.DataFrame()

# 녇도별 문서 추출 후 키워드 빈도수 분석

for year in tqdm_notebook(range(int(start_year),int(end_year)+1)):
    start_date = str(year) + "-01-01"
    end_date = str(year) + "-12-31"
    dt_index = (df['일자'] >= start_date) & (df['일자'] <= end_date)
    temp_df = df[dt_index]

    corpus = []
    for keywords in temp_df['특성추출(가중치순 상위 50개)']:
        corpus.append(keywords)

    # 각 년도별 corpus 정제
    cleaned_corpus = preprocess_corpus(corpus)

    vect = CountVectorizer(stop_words = stop_words, ngram_range=(1, 1), min_df= 0.005, max_df= 0.8)
    X = vect.fit_transform(cleaned_corpus)
    X = TfidfTransformer().fit_transform(X)

    count = X.toarray().sum(axis=0)
    idx = np.argsort(-count)
    count = count[idx]

    feature_name = np.array(vect.get_feature_names_out())[idx]

    result = list(zip(feature_name, count))
    result = pd.DataFrame(result)
    result.columns=[str(year) + ' 키워드','빈도수']
    result_df = pd.concat([result_df,result],axis=1)

result_df.head(50)

  0%|          | 0/15 [00:00<?, ?it/s]

,2010 키워드,빈도수,2011 키워드,빈도수,2012 키워드,빈도수,2013 키워드,빈도수,2014 키워드,빈도수,...,2020 키워드,빈도수,2021 키워드,빈도수,2022 키워드,빈도수,2023 키워드,빈도수,2024 키워드,빈도수
0,한국,13674.915089,서울,14070.113731,서울,16796.007926,전문가,20067.396315,전문가,24130.981004,...,온라인,42610.974583,온라인,41494.071675,서울,31567.451194,한국,39216.011790,서울,14178.838529
1,서울,13440.282548,한국,13496.432716,한국,14942.231832,서울,18295.031945,한국,21220.990525,...,서울,30524.992145,서울,29207.086839,한국,29536.170354,서울,37617.579629,ai,12834.536189
2,미국,10195.655412,미국,11001.856980,전문가,14417.791125,한국,17094.704893,서울,21058.258233,...,미국,27891.829795,미국,29169.552122,온라인,27969.980352,미국,32444.381871,한국,11778.027552
3,중국,7810.213856,일본,8157.236055,미국,11944.794239,미국,14013.675055,미국,15501.954868,...,한국,27433.904696,한국,28225.836395,미국,26357.447750,온라인,24853.307000,미국,11683.628152
4,온라인,6856.839818,중국,7792.258749,중국,9023.883760,중국,10724.958105,중국,13953.795927,...,코로나,26739.375580,코로나,22391.295322,전문가,18331.300561,ai,24136.811906,온라인,9824.455770
5,우리나라,6737.244968,전문가,7728.183885,편의점,8673.571557,편의점,9018.914256,온라인,13332.578420,...,빅데이터,21084.079768,빅데이터,18648.899558,지자체,16297.778623,전문가,20263.969395,지자체,7782.629733
6,일본,6416.993338,소비자,7315.887267,소비자,7746.923311,온라인,8944.073161,편의점,10242.656288,...,코로나바이러스,19994.583162,전문가,18580.568073,빅데이터,15692.106434,지자체,19945.423428,전문가,7593.282487
7,캠페인,6270.436894,편의점,6969.291420,캠페인,7663.975939,소비자,8498.203297,일본,10067.263954,...,전문가,18514.612329,소비자,16547.648882,소비자,14680.352271,일본,18341.967124,소비자,6199.046719
8,전문가,6246.220644,우리나라,6959.294752,온라인,7567.783252,일본,8456.794672,소비자,9363.975967,...,중국,16775.562673,지자체,16289.429318,편의점,14337.488997,중국,16968.131461,중국,5843.536718
9,소비자,5985.532501,캠페인,6938.042347,일본,7394.376166,캠페인,8053.942756,캠페인,8686.900367,...,소비자,15992.063680,ai,15472.865114,ai,14133.411871,편의점,15990.845969,빅데이터,5777.087730


In [14]:
result_df.to_excel('년도별 출현단어.xlsx', index=False)

## **2. 전체년도의 출현단어**



In [11]:
# 시작년도, 끝년도 지정
start_year = 2010
end_year = 2024

corpus = [""] * len(df)

result_df = pd.DataFrame()

# 녇도별 문서 추출 후 키워드 빈도수 분석

start_date = str(year) + "-01-01"
end_date = str(year) + "-12-31"
dt_index = (df['일자'] >= start_date) & (df['일자'] <= end_date)
temp_df = df[dt_index]

corpus = []
for keywords in temp_df['특성추출(가중치순 상위 50개)']:
    corpus.append(keywords)

# 각 년도별 corpus 정제
cleaned_corpus = preprocess_corpus(corpus)

vect = CountVectorizer(stop_words = stop_words, ngram_range=(1, 1), min_df= 0.005, max_df= 0.8)
X = vect.fit_transform(cleaned_corpus)
X = TfidfTransformer().fit_transform(X)

count = X.toarray().sum(axis=0)
idx = np.argsort(-count)
count = count[idx]

feature_name = np.array(vect.get_feature_names_out())[idx]

result = list(zip(feature_name, count))
result = pd.DataFrame(result)
result.columns=['출현단어_키워드','빈도수']
result_df = pd.concat([result_df,result],axis=1)

result_df.to_excel('전체년도별 출현단어.xlsx', index=False)

result_df

,출현단어_키워드,빈도수
0,서울,14178.838529
1,ai,12834.536189
2,한국,11778.027552
3,미국,11683.628152
4,온라인,9824.455770
...,...,...
106,식재료,810.599565
107,기부,739.576689
108,조성,705.330286
109,청소년,679.217631


## **3. 종합계획 차수별 출현단어**
- 어린이 식생활안전관리 종합계획

> 1차: 2010-2012

> 2차: 2013-2015

> 3차: 2016-2018

> 4차: 2019-2021

> 5차: 2022-2024

In [12]:
# 시작년도, 끝년도 지정
start_year = 2010
end_year = 2024

corpus = [""] * len(df)

result_df = pd.DataFrame()

# 녇도별 문서 추출 후 키워드 빈도수 분석


# Define the ranges and their names
year_ranges = [
    (2010, 2012, "1차: 2010-2012"),
    (2013, 2015, "2차: 2013-2015"),
    (2016, 2018, "3차: 2016-2018"),
    (2019, 2021, "4차: 2019-2021"),
    (2022, 2024, "5차: 2022-2024")
]

In [13]:
# List to store individual DataFrames for each period
period_dataframes = []

# Iterate over each range
for start_year, end_year, name in year_ranges:
    print(name)
    corpus = []

    for year in tqdm_notebook(range(start_year, end_year + 1)):
        start_date = f"{year}-01-01"
        end_date = f"{year}-12-31"

        dt_index = (df['일자'] >= start_date) & (df['일자'] <= end_date)
        temp_df = df[dt_index]

        for keywords in temp_df['특성추출(가중치순 상위 50개)']:
            corpus.append(keywords)

    # 각 년도별 corpus 정제
    cleaned_corpus = preprocess_corpus(corpus)

    vect = CountVectorizer(stop_words=stop_words, ngram_range=(1, 1), min_df=0.005, max_df=0.8)
    X = vect.fit_transform(cleaned_corpus)
    X = TfidfTransformer().fit_transform(X)

    count = X.toarray().sum(axis=0)
    idx = np.argsort(-count)
    count = count[idx]

    feature_name = np.array(vect.get_feature_names_out())[idx]

    result = list(zip(feature_name, count))
    result = pd.DataFrame(result)
    result.columns = ['키워드', '빈도수']
    result['기간'] = name

    # Append result to the list
    period_dataframes.append((result, name))

    display(result)

# Save all period dataframes into a single Excel file with multiple sheets
with pd.ExcelWriter('종합계획차수별_출현단어_빈도.xlsx') as writer:
    for result, name in period_dataframes:
        safe_sheet_name = name.replace(":", " ").replace("/", " ")
        result.to_excel(writer, sheet_name=safe_sheet_name, index=False)

# Optionally, display the combined result
result_df = pd.concat([df for df, name in period_dataframes], ignore_index=True)
display(result_df)


1차: 2010-2012


  0%|          | 0/3 [00:00<?, ?it/s]

,키워드,빈도수,기간
0,서울,44809.432981,1차: 2010-2012
1,한국,42635.555090,1차: 2010-2012
2,미국,33530.175104,1차: 2010-2012
3,전문가,28703.482001,1차: 2010-2012
4,중국,24849.994188,1차: 2010-2012
...,...,...,...
116,대학,1976.107389,1차: 2010-2012
117,조성,1918.883551,1차: 2010-2012
118,농림수산식품부,1904.974012,1차: 2010-2012
119,관내,1823.035532,1차: 2010-2012


2차: 2013-2015


  0%|          | 0/3 [00:00<?, ?it/s]

...

,키워드,빈도수,기간
0,온라인,102302.030984,4차: 2019-2021
1,서울,92814.205354,4차: 2019-2021
2,한국,87370.971594,4차: 2019-2021
3,미국,81234.467338,4차: 2019-2021
4,빅데이터,58985.100167,4차: 2019-2021
...,...,...,...
104,어린이,5254.700979,4차: 2019-2021
105,확산,5238.817301,4차: 2019-2021
106,생활,5149.111385,4차: 2019-2021
107,기부,4894.744159,4차: 2019-2021


5차: 2022-2024


  0%|          | 0/3 [00:00<?, ?it/s]

,키워드,빈도수,기간
0,서울,84284.697749,5차: 2022-2024
1,한국,81226.248550,5차: 2022-2024
2,미국,71124.122389,5차: 2022-2024
3,온라인,63145.966978,5차: 2022-2024
4,ai,51268.888045,5차: 2022-2024
...,...,...,...
102,점검,4961.065419,5차: 2022-2024
103,시민,4894.786918,5차: 2022-2024
104,학부모들,4492.591765,5차: 2022-2024
105,기부,4491.339292,5차: 2022-2024


,키워드,빈도수,기간
0,서울,44809.432981,1차: 2010-2012
1,한국,42635.555090,1차: 2010-2012
2,미국,33530.175104,1차: 2010-2012
3,전문가,28703.482001,1차: 2010-2012
4,중국,24849.994188,1차: 2010-2012
...,...,...,...
553,점검,4961.065419,5차: 2022-2024
554,시민,4894.786918,5차: 2022-2024
555,학부모들,4492.591765,5차: 2022-2024
556,기부,4491.339292,5차: 2022-2024


## **END**